In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KernelDensity
from scipy.stats import gamma

In [2]:
# import the csv file
data = pd.read_csv('IncPeriodData.csv')

In [3]:
# show head of the csv file
data.head()

,IncPeriod
0,0.624314
1,9.814797
2,4.975259
3,2.687350
4,11.322319


In [4]:
# histogram of data
# you can use the pandas method dataframe.hist(bins=nmbr_bins)
# specify appropriate number of bins



# Data Cleaning

In [5]:
# drop the lines from the DataFrame that exceed a certain value
# see method: dataframe.drop(datataframe[dataframe.column > value].index, inplace=True)



In [6]:
# check again histogram of data



In [7]:
# print mean, median, std of sample
# use again the pandas functions: dataframe.mean(), ...



# Kernel Density Estimation

In [8]:
# you can use KernelDensity(kernel=kernel, bandwidth=value)
# create instance from kernel density estimator, specify kernel and bandwith
# see https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity.sample



In [9]:
# fit the kernel density estimator: instance.fit([...])
# attention: the fit() function wants a shape [nmbr_samples, nmbr_features] array
# -- > best reshape your data, e.g. with np.array(data.IncPeriod).reshape(-1,1)



In [10]:
# plot the kernel density probability distribution function: 
# instance.score_samples() returns the log-pdf (i.e. you still need to apply np.exp(...))
# attention: the score_samples() function also wants a shape [nmbr_samples, nmbr_features] array
# -- > best reshape your x-value grid, e.g. with np.array(x).reshape(-1,1) or x = np.linspace(0,max_val)[:, np.newaxis]



In [11]:
# generate samples from estimated density function:
# instance.sample(sample_size) returns a sample of size sample_size
# then create new instance of KernelDensity, fit to the sample and plot again the pdf



# Distribution Folding

In [12]:
# define the new probability density function as a function
# you probabily want to use the scipy.stats.gamma.pdf(x,a=4) function

def folded_pdf(x):
    """Gets an x-grid and returns the folded probability distribution

    Parameters
    ----------
    x : numpy array of shape (N,1)
        The x-grid on which the probability densitiy function is evaluated

    Returns
    -------
    numpy array of shape (N,1)
        The resulting, folded pdf
    """

    # the gamma distribution
    g = ...
    # the pdf of the kernel density estimation
    p = ...
    # convolve
    c = np.convolve(g,p)
    # truncate the result for same length with grid
    c = c[:len(x)]
    
    return c

In [13]:
# plot the folded pdf

x = np.linspace(0,30)[:, np.newaxis]

...